In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

from deep_image_prior.utils.denoising_utils import *

from deep_image_prior.src.skip_unet import SkipUnet

In [ ]:
imsize =-1
PLOT = True
sigma = 25
sigma_ = sigma/255.

In [ ]:
fname = '../denoising/F16_GT.png'

In [ ]:
if fname == '../denoising/F16_GT.png':
    # Add synthetic noise
    img_pil = crop_image(get_image(fname, imsize)[0], d=32)
    img_np = pil_to_np(img_pil)
    
    img_noisy_pil, img_noisy_np = get_noisy_image(img_np, sigma_)

In [ ]:
plt.imshow(img_pil)

In [ ]:
print(img_noisy_np.shape)

In [ ]:
model = SkipUnet()
adam = tf.keras.optimizers.Adam(learning_rate=1e-2, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
model.compile(optimizer=adam, loss=tf.keras.losses.MeanSquaredError())
model.build((1, 512, 512, 32))

In [ ]:
model.summary()

In [ ]:
# net_input = get_noise(32, 'noise', (img_pil.size[1], img_pil.size[0])).type(dtype).detach()
net_input = tf.random.uniform([1, 512, 512, 32], minval=0, maxval=1/10)

In [ ]:
img_noisy_np = img_noisy_np.transpose(1, 2, 0)

In [ ]:
img_noisy_np = img_noisy_np[np.newaxis, ...]

In [ ]:
plt.imshow(img_noisy_np.squeeze())

In [ ]:
images = []

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 50 == 0:
            pred = model.predict(net_input).squeeze()
            images.append((pred, epoch))
#             pred = (pred - np.min(pred))/(np.max(pred) - np.min(pred))
            
#             plt.imshow(pred)
#             plt.show()

In [ ]:
model.fit(x=net_input, y=img_noisy_np, epochs=4000, callbacks=[CustomCallback()], verbose=0)

In [ ]:
pred = model.predict(net_input).squeeze()

In [ ]:
plt.imshow(pred)

In [ ]:
plt.imshow(img_noisy_pil)

In [ ]:
fig=plt.figure(figsize=(40, 5 * len(images)))
rows, columns = 15, 2
for i in range(0, 15):
    img, epoch = images[2*i+40]
    
#     # Plot ground truth
#     s = fig.add_subplot(rows, columns, columns*i+1)
#     s.set_title("Ground truth", size=15)
#     plt.imshow(img_pil)
        
    # Plot prediction
    s = fig.add_subplot(rows, columns, columns*i+1)
    s.set_title("Epoch {}".format(epoch), size=40)
    plt.imshow(img)
    
    # Plot noisy image
    s = fig.add_subplot(rows, columns, columns*i+2)
    s.set_title("Noisy image", size=40)
    plt.imshow(img_noisy_pil)


    
plt.savefig('Denoising_deep_image_prior.pdf')

In [ ]:
for img, epoch in images:
    plt.imshow(img)
    plt.title("Epoch {}".format(epoch))
    plt.show()

In [ ]:
plt.imshow(img_noisy_pil)

In [ ]:
INPUT = 'noise' # 'meshgrid'
pad = 'reflection'
OPT_OVER = 'net' # 'net,input'

reg_noise_std = 1./30. # set to 1./20. for sigma=50
LR = 0.01

OPTIMIZER='adam' # 'LBFGS'
show_every = 100
exp_weight=0.99

if fname == '../denoising/F16_GT.png':
    print('hi')
    num_iter = 2400
    input_depth = 3
    figsize = 5 
    
    net = skip(
                input_depth, 3, 
                num_channels_down = [8, 16, 32, 64, 128], 
                num_channels_up   = [8, 16, 32, 64, 128],
                num_channels_skip = [0, 0, 0, 4, 4], 
                upsample_mode='bilinear',
                need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU')

    net = net.type(dtype)